In [1]:
import os
from datetime import datetime, timedelta
from glob import glob

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray
from dask.diagnostics import ProgressBar
from dask.distributed import Client, progress
from tqdm import tqdm

In [10]:
client = Client(n_workers=12)

/scratch/snx3000/kschuurm/lightning/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35145 instead
  warnings.warn(


In [ ]:
hres = xarray.open_zarr(
    "/scratch/snx3000/acarpent/EumetsatData/SEVIRI_WGS_2016-2022_RSS.zarr"
).drop_duplicates(dim="time")
hres = hres.rename_dims({"x": "lon", "y": "lat"}).rename_vars({"x": "lon", "y": "lat"})
hres = hres.drop_duplicates(dim="time")
hres

In [ ]:
sarah = xarray.open_zarr('/scratch/snx3000/kschuurm/ZARR/SARAH3.zarr')
sarah.SID.isel(time=160).plot.imshow()

In [ ]:
lat = hres.lat.copy()
lon = hres.lon.copy()
# del hres



def clean_reindex_sarah3(df):
    df = df.sel(time=df.record_status == 0).drop_vars(
        ["record_status", "lat_bnds", "lon_bnds"]
    )
    # df = clean(df)
    return df.reindex(lat=lat, lon=lon, method="nearest")


def clean_reindex_sarah3_easy(df):
    df = df.sel(time=df.record_status == 0).drop_vars(
        ["record_status", "lat_bnds", "lon_bnds"]
    )
    # df = df.isel(time=timenotnan)
    return df.reindex(lat=lat, lon=lon, method="nearest")

In [ ]:
fns = glob("/scratch/snx3000/kschuurm/DATA/SARAH3/SID/SIDin*.nc")

sid = xarray.open_mfdataset(
    fns,
    parallel=True,
    engine="h5netcdf",
    chunks={"time": 48, "lat": -1, "lon": -1},
)
# preprocess=clean_reindex_sarah3)
sid

In [ ]:
sid = clean_reindex_sarah3(sid).chunk({"time": 48, "lat": -1, "lon": -1})

In [ ]:
a = sid_slice.SID.to_zarr("/scratch/snx3000/kschuurm/ZARR/SARAH3.zarr", mode="a")
del a, sid

In [ ]:
fns = glob("/scratch/snx3000/kschuurm/DATA/SARAH3/SIS/SISin*.nc")

sis = xarray.open_mfdataset(
    fns,
    parallel=True,
    engine="h5netcdf",
    chunks={"time": 48, "lat": -1, "lon": -1},
)

In [ ]:
sis = clean_reindex_sarah3(sis).chunk({"time": 48, "lat": -1, "lon": -1})

In [ ]:
a = sis.SIS.to_zarr("/scratch/snx3000/kschuurm/ZARR/SARAH3.zarr", mode="a")
del a, sis

In [ ]:
fns = glob("/scratch/snx3000/kschuurm/DATA/SARAH3/CAL/CALin*.nc")

cal = xarray.open_mfdataset(
    fns,
    parallel=True,
    engine="h5netcdf",
    chunks={"time": 48, "lat": -1, "lon": -1},
)
cal

In [ ]:
cal = clean_reindex_sarah3(cal).chunk({"time": 48, "lat": -1, "lon": -1})

In [ ]:
a = cal.CAL.to_zarr("/scratch/snx3000/kschuurm/ZARR/SARAH3_new.zarr", mode="a")


In [ ]:
SARAH = xarray.open_zarr("/scratch/snx3000/kschuurm/ZARR/SARAH3_new.zarr")

In [ ]:
import dask


def cal_to_ki(x):
    first_cond = x <= -0.05
    second_cond = (x > -0.05) & (x <= 0.8)
    third_cond = (x > 0.8) & (x <= 1.05)
    fourth_cond = (x > 1.05) & (x < 1.2)
    fifth_cond = x == 1.2

    y = dask.array.zeros(shape=x.shape)

    def func1(x):
        return 1 - x

    def func2(x):
        return 1.1661 - 1.781 * x + 0.73 * x**2

    funcs = [1.05, func1, func2, 0.09, 0]
    conds = [first_cond, second_cond, third_cond, fourth_cond, fifth_cond]

    # y[first_cond] = 1.05
    # y[second_cond] = 1 - x[second_cond]
    # y[third_cond] = 1.1661 - 1.781*x[third_cond] + 0.73*x[third_cond]**2
    # y[fourth_cond] = 0.09
    # y[fifth_cond] = 0
    return dask.array.piecewise(x, conds, funcs)


def cal_to_ki2(x):
    y = np.nan
    if x <= -0.05:
        y = 1.05
    elif x > -0.05 & x <= 0.8:
        y = 1 - x
    elif x > 0.8 & x <= 1.05:
        y = 1.1661 - 1.781 * x + 0.73 * x**2
    elif x > 1.05 & x < 1.2:
        y = 0.09
    elif x == 1.2:
        y = 0
    return y

In [ ]:
ki = xarray.apply_ufunc(cal_to_ki, SARAH.CAL, dask="allowed")

In [ ]:
ki = ki.rename("KI")

In [ ]:
ki = ki.assign_attrs(SARAH.CAL.attrs)
ki.attrs["long_name"] = "Clear Sky Index"

In [ ]:
ki.to_zarr("/scratch/snx3000/kschuurm/ZARR/SARAH3.zarr", mode="a")

In [11]:
SARAH2 = xarray.open_zarr("/scratch/snx3000/kschuurm/ZARR/SARAH3_new.zarr")
SARAH = xarray.open_zarr("/scratch/snx3000/acarpent/SARAH3_2016-2022.zarr")

In [8]:
SARAH2

<xarray.Dataset>
Dimensions:       (channel: 4, time: 139915, lat: 658, lon: 736)
Coordinates:
  * channel       (channel) object 'CAL' 'KI' 'SID' 'SIS'
  * lat           (lat) float32 28.98 29.02 29.08 29.12 ... 61.72 61.78 61.83
  * lon           (lon) float32 -7.775 -7.725 -7.675 ... 28.88 28.92 28.98
  * time          (time) datetime64[ns] 2015-01-01 ... 2023-01-01T23:30:00
Data variables:
    channel_data  (channel, time, lat, lon) float16 dask.array<chunksize=(4, 1, 658, 736), meta=np.ndarray>
    max_x         (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    max_y         (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    min_x         (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    min_y         (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
Attributes: (12/41)
    CDI:                        Climate Data Interface version 1.9.10 (https:...
    CDO:                        Climate Data Operators version 1.9.10 (https:...
    Conventions:                CF-1.7,ACDD-1.3
    creator_email:              contact.cmsaf@dwd.de
    creator_name:               DE/DWD
    creator_type:               institution
    ...                         ...
    time_coverage_duration:     P1D
    time_coverage_end:          2023-01-01T00:00:00
    time_coverage_resolution:   PT30M
    time_coverage_start:        2022-12-31T00:00:00
    title:                      CM SAF Surface Solar Radiation Climate Data R...
    variable_id:                CAL

In [14]:
max_x, max_y, min_x, min_y = [SARAH[nm].data.persist() for nm in ['max_x', 'max_y', 'min_x', 'min_y']]

In [51]:
ds = xarray.Dataset(data_vars={'max_x':('time', max_x.compute()), 
                     'max_y':('time', max_y.compute()), 
                     'min_x':('time', min_x.compute()), 
                     'min_y':('time', min_y.compute())},
                    coords={'time':SARAH.time})

In [52]:
pixel_count = (max_y - min_y)*(max_x - min_x)
ds = ds.assign(pixel_count=('time', pixel_count))
ds

<xarray.Dataset>
Dimensions:      (time: 58115)
Coordinates:
  * time         (time) datetime64[ns] 2016-01-01T07:30:00 ... 2022-12-31T14:...
Data variables:
    max_x        (time) int64 736 736 736 736 736 736 ... 676 636 576 516 456
    max_y        (time) int64 338 418 478 518 558 598 ... 598 558 498 438 378
    min_x        (time) int64 320 240 180 140 100 60 20 0 0 ... 0 0 0 0 0 0 0 0
    min_y        (time) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    pixel_count  (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>

In [53]:
ds = ds.reindex(time=SARAH2.time, fill_value=-1).compute()
ds = ds.chunk({'time':-1}).compute()

/scratch/snx3000/kschuurm/lightning/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 18.15 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [54]:
ds = ds.rename_vars({'max_x':'max_lon', 'max_y':'max_lat', 'min_x':'min_lon', 'min_y':'min_lat'})
ds

<xarray.Dataset>
Dimensions:      (time: 139915)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01 ... 2023-01-01T23:30:00
Data variables:
    max_lon      (time) int64 -1 -1 -1 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1 -1 -1
    max_lat      (time) int64 -1 -1 -1 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1 -1 -1
    min_lon      (time) int64 -1 -1 -1 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1 -1 -1
    min_lat      (time) int64 -1 -1 -1 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1 -1 -1
    pixel_count  (time) int64 -1 -1 -1 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1 -1 -1

In [55]:
ds.to_zarr("/scratch/snx3000/kschuurm/ZARR/SARAH3_bnds.zarr", mode='w')

In [60]:
ds.isel(time=ds.pixel_count != -1).pixel_count.sum()

<xarray.DataArray 'pixel_count' ()>
array(23895353480)

In [ ]:
import concurrent
from functools import partial
from utils.etc import benchmark

def check(x):
    return np.isnan(x).any()


def check_all(x):
    return np.isnan(x).all()


def check_sum(x):
    return np.isnan(x).sum()


def compute_bnds(x):
    i = 0
    j = 0
    i_max = 740
    j_max = 662
    step = 20
    angle_step = j_max // 2
    if not check_all(x):
        c = check(x)
        if c:
            c_ul = check_sum(
                x.isel(
                    lon=slice(i, i + angle_step), lat=slice(j_max - angle_step, j_max)
                )
            )
            c_ur = check_sum(
                x.isel(
                    lon=slice(i_max - angle_step, i_max),
                    lat=slice(j_max - angle_step, j_max),
                )
            )
            c_ll = check_sum(
                x.isel(lon=slice(i, i + angle_step), lat=slice(j, j + angle_step))
            )
            c_lr = check_sum(
                x.isel(
                    lon=slice(i_max - angle_step, i_max), lat=slice(j, j + angle_step)
                )
            )

            if c_ul > c_ur and c_ul > c_ll and c_ul > c_lr:
                c = check(x.isel(lon=slice(i, i_max), lat=slice(j, j_max)))
                while c:
                    j_max -= step
                    i += step
                    c = check(x.isel(lon=slice(i, i_max), lat=slice(j, j_max)))

            elif c_ur > c_ul and c_ur > c_ll and c_ur > c_lr:
                c = check(x.isel(lon=slice(i, i_max), lat=slice(j, j_max)))
                while c:
                    i_max -= step
                    j_max -= step
                    c = check(x.isel(lon=slice(i, i_max), lat=slice(j, j_max)))

            elif c_ll > c_ul and c_ll > c_ur and c_ll > c_lr:
                c = check(x.isel(lon=slice(i, i_max), lat=slice(j, j_max)))
                while c:
                    i += step
                    j += step
                    c = check(x.isel(lon=slice(i, i_max), lat=slice(j, j_max)))

            elif c_lr > c_ul and c_lr > c_ur and c_lr > c_ll:
                c = check(x.isel(lon=slice(i, i_max), lat=slice(j, j_max)))
                while c:
                    i_max -= step
                    j += step
                    c = check(x.isel(lon=slice(i, i_max), lat=slice(j, j_max)))
            else:
                return None
    else:
        return None

    if i_max > i + 128 and j_max > j + 128:
        return i, j, i_max, j_max
    else:
        return None

def compute_minmax_bnds(x, j):
    bnd = compute_bnds(x[j])
    if bnd is not None:
        if bnd[2] > bnd[0] + 128 and bnd[3] > bnd[1] + 128:
            return j, bnd[0], bnd[1], bnd[2], bnd[3]
    else:
        return None

def clean(x):
    # remove unreliable estimations
    x = x.sel(time=x.record_status==0)
    
    # compute indeces of images with less than 50% nans
    img_x_dim = x.lon.shape[0]
    img_y_dim = x.lat.shape[0]
    idx = np.where(x.CAL.isnull().sum(dim=('lat','lon'))/(img_x_dim*img_y_dim) < .5)[0]
    
    # initialize indices lists
    final_idx = []
    min_x_lst = []
    min_y_lst = []
    max_x_lst = []
    max_y_lst = []

    # compute image boundaries for every admitted index
    for j in tqdm(idx):
        bnd = compute_bnds(x.CAL[j])
        if bnd is not None:
            if bnd[2]>bnd[0]+128 and bnd[3]>bnd[1]+128:
                final_idx.append(j)
                min_x_lst.append(bnd[0])
                min_y_lst.append(bnd[1])
                max_x_lst.append(bnd[2])
                max_y_lst.append(bnd[3])
                
    x = x.isel(time=final_idx)
    
    x['min_x'] = ('time', min_x_lst)
    x['min_y'] = ('time', min_y_lst)
    x['max_x'] = ('time', max_x_lst)
    x['max_y'] = ('time', max_y_lst)
    x = x[['CAL', 'min_x', 'min_y', 'max_x', 'max_y']]
    return x

